In [ ]:
import os

In [ ]:
!git clone https://github.com/terryluan12/Genrify.git

In [ ]:
import sys
sys.path.insert(0, "content/Genrify/src")

In [ ]:
from datasources import download_datasets
from preprocessing import preprocess

In [ ]:
if not os.path.isdir("content/Genrify/datasources/Data"):
    download_datasets("content/Genrify/")

preprocess("content/Genridfy/")